In [9]:
import win32gui
import win32con
import win32api

In [39]:
def EnumWindows():
    windows = {}
    def _MyCallback( hwnd, extra ):
        windows = extra
        temp={}
        temp['hex'] = hex(hwnd)
        temp['ClassName'] = win32gui.GetClassName(hwnd)
        temp['WindowText'] = win32gui.GetWindowText(hwnd)
        windows[hwnd] = temp
    win32gui.EnumWindows(_MyCallback, windows)
    return windows

In [16]:
def EnumChildWindows(hwnd):
    windows = {}
    def _MyCallback( hwnd, extra ):
        windows = extra
        temp={}
        temp['hex'] = hex(hwnd)
        temp['ClassName'] = win32gui.GetClassName(hwnd)
        temp['WindowText'] = win32gui.GetWindowText(hwnd)
        windows[hwnd] = temp
    win32gui.EnumChildWindows(hwnd,_MyCallback, windows)
    return windows

In [21]:
childs = EnumChildWindows('1444364')
childs

{395806: {'hex': '0x60a1e',
  'ClassName': 'ApplicationFrameTitleBarWindow',
  'WindowText': ''},
 854514: {'hex': '0xd09f2',
  'ClassName': 'ApplicationFrameTitleBarWindow',
  'WindowText': ''},
 526886: {'hex': '0x80a26',
  'ClassName': 'ApplicationFrameInputSinkWindow',
  'WindowText': ''}}

In [30]:
child_text = win32gui.GetWindowText('1444364')
print(child_text)

设置


In [50]:
# 搜索子窗口
# 枚举子窗口
hwndChildList = []
win32gui.EnumChildWindows('1444364', lambda hwnd, param: param.append(hwnd),  hwndChildList)
hwndChildList

[395806, 854514, 526886]

In [87]:
import cv2
import numpy as np
from ctypes import windll
import win32gui
import win32ui
 
 
def capture_win_alt(hwnd=None, program_windowTitle=None, program_className=None,img_name='temp.png'):
    # Adapted from https://stackoverflow.com/questions/19695214/screenshot-of-inactive-window-printwindow-win32gui
    if hwnd is None:
        windll.user32.SetProcessDPIAware()
        windows = EnumWindows()
        for key,item in windows.items() :
            if item['ClassName'] == program_className:
                if program_windowTitle != None:
                    if program_windowTitle in item['WindowText'] or program_windowTitle == item['WindowText']:
                        program_windowTitle = item['WindowText']
                        hwnd = key
                        print(key,item)
                        break;
                    else:
                        hwnd = key
                        print(key,item)
                        break;
    if not hwnd:
        hwnd = win32gui.FindWindow(program_className, program_windowTitle)
        # hwnd = win32gui.FindWindowEx(0, 0, program_className, program_windowTitle)
    if not hwnd:
        return None
    left, top, right, bottom = win32gui.GetClientRect(hwnd)
    w = right - left
    h = bottom - top
    hwnd_dc = win32gui.GetWindowDC(hwnd)
    mfc_dc = win32ui.CreateDCFromHandle(hwnd_dc)
    save_dc = mfc_dc.CreateCompatibleDC()
    bitmap = win32ui.CreateBitmap()
    bitmap.CreateCompatibleBitmap(mfc_dc, w, h)
    save_dc.SelectObject(bitmap)
    # If Special K is running, this number is 3. If not, 1
    result = windll.user32.PrintWindow(hwnd, save_dc.GetSafeHdc(), 3)
    bmpinfo = bitmap.GetInfo()
    bmpstr = bitmap.GetBitmapBits(True)
    img = np.frombuffer(bmpstr, dtype=np.uint8).reshape((bmpinfo["bmHeight"], bmpinfo["bmWidth"], 4))
    img = np.ascontiguousarray(img)[..., :-1]  # make image C_CONTIGUOUS and drop alpha channel
    if not result:  # result should be 1
        win32gui.DeleteObject(bitmap.GetHandle())
        save_dc.DeleteDC()
        mfc_dc.DeleteDC()
        win32gui.ReleaseDC(hwnd, hwnd_dc)
        raise RuntimeError(f"Unable to acquire screenshot! Result: {result}")
    cv2.imwrite(img_name, img)
    return img,w,h,hwnd

In [94]:
from cnocr import CnOcr


img,w,h,hwnd = capture_win_alt(program_className='ApplicationFrameWindow',program_windowTitle='')

ocr = CnOcr() 
result = ocr.ocr(img)

# 具体参数设置参考：https://cnocr.readthedocs.io/zh/latest/usage/


723436 {'hex': '0xb09ec', 'ClassName': 'ApplicationFrameWindow', 'WindowText': '设置'}


In [96]:
print(w,h)

1282 1000


In [97]:
from cnocr import CnOcr

ocr = CnOcr()
img,x,y,hwnd = capture_win_alt(program_className='ApplicationFrameWindow',program_windowTitle='')

step_key = ['时间和语言']
text_positions = []
for item in ocr.ocr(img):
    if item['score'] > 0.9:
        if item['text'] in step_key:
            temp = {
                'text': item['text'],
                'position': [
                    (item['position'][0][0]+item['position'][2][0])/2, 
                    (item['position'][0][1]+item['position'][2][1])/2
                             ]
            }
            temp['position'][0] = temp['position'][0] / x
            temp['position'][1] = temp['position'][1] / y
            text_positions.append(temp)


723436 {'hex': '0xb09ec', 'ClassName': 'ApplicationFrameWindow', 'WindowText': '设置'}


In [98]:
left, top, right, bottom = win32gui.GetWindowRect(hwnd)
print(left, top, right, bottom)
cy = int(text_positions[0]['position'][1] * (bottom - top) + top)
cx = int(text_positions[0]['position'][0] * (right - left) + left)
print(cx,cy)
long_position = win32api.MAKELONG(cx, cy)#模拟鼠标指针 传送到指定坐标
win32api.SendMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, long_position)#模拟鼠标按下
win32api.SendMessage(hwnd, win32con.WM_LBUTTONUP, win32con.MK_LBUTTON, long_position)#模拟鼠标弹起

19 10 1317 1018
114 508


0

In [91]:
text_positions

[{'text': '时间和语言', 'position': array([     0.0945,     0.38612])}]

In [76]:
text_positions

[{'text': '时间和语言', 'position': array([      0.094,     0.38612])}]